In [8]:
import numpy as np

def jaya(dist_matrix, demands, capacity, pop_size=50, max_iter=2000):
    """
    Aplica Jaya para el CVRP. dist_matrix es NxN de distancias, demands es vector de tamaño N,
    capacity es carga máxima. Devuelve lista de rutas y costo total.
    """
    n = len(demands)
    customers = list(range(1, n))  # excluye el depósito 0
    # Inicializar población: cada solución es un vector real de tamaño (n-1)
    population = np.random.rand(pop_size, n-1)
    # Función interna para evaluar una solución contínua:
    def eval_sol(x):
        # mapear a permutación ordenando índices de x
        perm = [customers[i] for i in np.argsort(x)]
        # construir rutas por acumulación de demanda
        cost = 0.0
        current = 0
        load = 0.0
        for cust in perm:
            if load + demands[cust] > capacity:
                cost += dist_matrix[current, 0]  # volver al depósito
                current = 0
                load = 0.0
            cost += dist_matrix[current, cust]
            load += demands[cust]
            current = cust
        cost += dist_matrix[current, 0]
        return cost, perm
    # Evaluar población inicial
    costs = np.zeros(pop_size)
    for i in range(pop_size):
        costs[i], _ = eval_sol(population[i])
    # Iteraciones de Jaya
    for t in range(max_iter):
        # Identificar mejores/peores
        best_idx = np.argmin(costs)
        worst_idx = np.argmax(costs)
        best = population[best_idx].copy()
        worst = population[worst_idx].copy()
        # Actualizar cada individuo
        for i in range(pop_size):
            r1 = np.random.rand(n-1)
            r2 = np.random.rand(n-1)
            new = population[i] + r1*(best - np.abs(population[i])) - r2*(worst - np.abs(population[i]))
            # Opcional: truncar entre [0,1] para estabilidad
            new = np.clip(new, 0, 1)
            new_cost, _ = eval_sol(new)
            # Selección codiciosa
            if new_cost < costs[i]:
                population[i] = new
                costs[i] = new_cost
    # Retornar mejor solución encontrada
    final_cost = costs.min()
    final_sol = population[costs.argmin()]
    best_cost, best_perm = eval_sol(final_sol)
    return best_perm, best_cost


In [9]:
def gwo(dist_matrix, demands, capacity, num_wolves=30, max_iter=500):
    """
    Optimización de Lobo Gris para CVRP. Cada solución es un 'lobo' vector real.
    """
    n = len(demands)
    customers = list(range(1, n))
    # Inicializar lobos aleatorios
    wolves = np.random.rand(num_wolves, n-1)
    def eval_sol(x):
        perm = [customers[i] for i in np.argsort(x)]
        cost = 0.0
        current = 0
        load = 0.0
        for cust in perm:
            if load + demands[cust] > capacity:
                cost += dist_matrix[current, 0]
                current = 0
                load = 0.0
            cost += dist_matrix[current, cust]
            load += demands[cust]
            current = cust
        cost += dist_matrix[current, 0]
        return cost, perm
    # Evaluar inicial
    fitness = np.array([eval_sol(w)[0] for w in wolves])
    for t in range(max_iter):
        # Ordenar lobos por fitness
        idx = np.argsort(fitness)
        alpha, beta, gamma = wolves[idx[0]], wolves[idx[1]], wolves[idx[2]]
        a = 2 * (1 - t/max_iter)  # coeficiente lineal decreciente
        # Actualizar cada lobo
        for i in range(num_wolves):
            r1, r2 = np.random.rand(n-1), np.random.rand(n-1)
            A1 = 2*a*r1 - a;    C1 = 2*r2
            D_alpha = np.abs(C1*alpha - wolves[i])
            X1 = alpha - A1*D_alpha
            r1, r2 = np.random.rand(n-1), np.random.rand(n-1)
            A2 = 2*a*r1 - a;    C2 = 2*r2
            D_beta = np.abs(C2*beta - wolves[i])
            X2 = beta - A2*D_beta
            r1, r2 = np.random.rand(n-1), np.random.rand(n-1)
            A3 = 2*a*r1 - a;    C3 = 2*r2
            D_gamma = np.abs(C3*gamma - wolves[i])
            X3 = gamma - A3*D_gamma
            new_wolf = (X1 + X2 + X3) / 3
            new_cost, _ = eval_sol(new_wolf)
            if new_cost < fitness[i]:
                wolves[i] = new_wolf
                fitness[i] = new_cost
    # Mejor lobo al final
    best_idx = fitness.argmin()
    best_cost, best_perm = eval_sol(wolves[best_idx])
    return best_perm, best_cost


In [10]:
def simulated_annealing(dist_matrix, demands, capacity, T0=1000, cooling_rate=0.99, max_iter=10000):
    """
    Recocido simulado para CVRP. Se representa la solución como permutación de clientes.
    """
    n = len(demands)
    customers = list(range(1, n))
    # Función para calcular costo dado un orden de clientes
    def eval_perm(perm):
        cost = 0.0; current = 0; load = 0.0
        for cust in perm:
            if load + demands[cust] > capacity:
                cost += dist_matrix[current, 0]
                current = 0; load = 0.0
            cost += dist_matrix[current, cust]
            load += demands[cust]
            current = cust
        cost += dist_matrix[current, 0]
        return cost
    # Solución inicial (aleatoria)
    current_perm = customers.copy()
    np.random.shuffle(current_perm)
    current_cost = eval_perm(current_perm)
    best_perm = current_perm.copy()
    best_cost = current_cost
    T = T0
    for i in range(max_iter):
        # Generar vecino: swap aleatorio
        i1, i2 = np.random.choice(len(customers), 2, replace=False)
        new_perm = current_perm.copy()
        new_perm[i1], new_perm[i2] = new_perm[i2], new_perm[i1]
        new_cost = eval_perm(new_perm)
        # Aceptar si mejor, o con probabilidad
        if new_cost < current_cost or np.random.rand() < np.exp((current_cost - new_cost)/T):
            current_perm, current_cost = new_perm, new_cost
            if new_cost < best_cost:
                best_perm, best_cost = new_perm.copy(), new_cost
        # Enfriar temperatura
        T *= cooling_rate
        if T < 1e-3:
            break
    return best_perm, best_cost


In [11]:
import numpy as np

capacity_truck = 15.0  # toneladas
container_capacity = 0.8  # toneladas (se usa para redondear demandas si es preciso)

distances_weekly = np.load('distance_matrix_semanal.npy')
demands_weekly  = 0.8*np.load('campanas_por_ubi_semanal.npy')
distances_biweekly = np.load('distance_matrix_quincenal.npy')
demands_biweekly  = 0.8*np.load('campanas_por_ubi_quincenal.npy')

In [12]:
np.shape(distances_weekly)

(38, 38)

In [13]:
demands_weekly

array([ 6.4,  6.4,  4.8,  4.8, 11.2,  1.6, 11.2,  4.8,  4.8,  1.6,  1.6,
        4. ,  4. ])

In [14]:
np.shape(distances_biweekly)

(39, 39)

In [15]:
demands_biweekly

array([ 5.6,  3.2, 13.6,  3.2,  3.2,  5.6,  7.2, 12. ,  6.4,  4.8])

In [16]:
# Ejecutar Jaya para rutas semanales
perm_jaya_weekly, cost_jaya_weekly = jaya(distances_weekly, demands_weekly, capacity_truck)
print("Jaya Semanal:", cost_jaya_weekly, "con orden:", perm_jaya_weekly)

# Ejecutar GWO para rutas quincenales
perm_gwo_biweekly, cost_gwo_biweekly = gwo(distances_biweekly, demands_biweekly, capacity_truck)
print("GWO Quincenal:", cost_gwo_biweekly, "con orden:", perm_gwo_biweekly)

# Ejecutar Simulated Annealing para rutas semanales
perm_sa_weekly, cost_sa_weekly = simulated_annealing(distances_weekly, demands_weekly, capacity_truck)
print("SA Semanal:", cost_sa_weekly, "con orden:", perm_sa_weekly)


Jaya Semanal: 85.16142483764757 con orden: [11, 9, 10, 12, 6, 3, 7, 2, 4, 8, 1, 5]
GWO Quincenal: 70.87147585756317 con orden: [4, 3, 1, 9, 5, 6, 7, 8, 2]
SA Semanal: 87.60969703022688 con orden: [1, 8, 11, 9, 12, 10, 2, 3, 7, 4, 5, 6]


In [17]:
import time

def construir_rutas_con_salida(perm, demands, capacity, distances):
    rutas = []
    ruta_actual = [0]
    carga = 0.0
    distancia_total = 0.0
    nodo_anterior = 0

    for cliente in perm:
        demanda = demands[cliente]
        if carga + demanda > capacity:
            ruta_actual.append(0)
            # Calcular distancia de la ruta
            dist = sum(distances[ruta_actual[i]][ruta_actual[i+1]] for i in range(len(ruta_actual)-1))
            distancia_total += dist
            rutas.append(ruta_actual)
            # Iniciar nueva ruta
            ruta_actual = [0, cliente]
            carga = demanda
            nodo_anterior = cliente
        else:
            ruta_actual.append(cliente)
            carga += demanda
            nodo_anterior = cliente

    # Agregar última ruta
    if ruta_actual[-1] != 0:
        ruta_actual.append(0)
        dist = sum(distances[ruta_actual[i]][ruta_actual[i+1]] for i in range(len(ruta_actual)-1))
        distancia_total += dist
        rutas.append(ruta_actual)

    return rutas, distancia_total


# semanal

## jaya

In [18]:
start = time.time()
perm, _ = jaya(distances_weekly, demands_weekly, capacity_truck)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764757
Camión 1: [0, 11, 9, 12, 10, 0]
Camión 2: [0, 4, 0]
Camión 3: [0, 6, 0]
Camión 4: [0, 1, 5, 8, 0]
Camión 5: [0, 7, 3, 2, 0]
Tiempo de ejecución: 5.25741171836853 segundos


In [19]:
start = time.time()
perm, _ = jaya(distances_weekly, demands_weekly, capacity_truck, pop_size=100, max_iter=2000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764759
Camión 1: [0, 6, 0]
Camión 2: [0, 7, 3, 2, 0]
Camión 3: [0, 8, 5, 1, 0]
Camión 4: [0, 4, 0]
Camión 5: [0, 12, 10, 9, 11, 0]
Tiempo de ejecución: 10.151625633239746 segundos


In [20]:
start = time.time()
perm, _ = jaya(distances_weekly, demands_weekly, capacity_truck,pop_size=150, max_iter=2000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764757
Camión 1: [0, 10, 12, 9, 11, 0]
Camión 2: [0, 6, 0]
Camión 3: [0, 8, 1, 5, 0]
Camión 4: [0, 4, 0]
Camión 5: [0, 3, 7, 2, 0]
Tiempo de ejecución: 15.591084718704224 segundos


## GW

In [21]:
start = time.time()
perm, _ = gwo(distances_weekly, demands_weekly, capacity_truck)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764759
Camión 1: [0, 9, 11, 10, 12, 0]
Camión 2: [0, 3, 7, 2, 0]
Camión 3: [0, 8, 5, 1, 0]
Camión 4: [0, 4, 0]
Camión 5: [0, 6, 0]
Tiempo de ejecución: 1.4312148094177246 segundos


In [22]:
start = time.time()
perm, _ = gwo(distances_weekly, demands_weekly, capacity_truck, num_wolves=50, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764759
Camión 1: [0, 12, 10, 11, 9, 0]
Camión 2: [0, 2, 7, 3, 0]
Camión 3: [0, 4, 0]
Camión 4: [0, 6, 0]
Camión 5: [0, 8, 5, 1, 0]
Tiempo de ejecución: 2.5935535430908203 segundos


In [23]:
start = time.time()
perm, _ = gwo(distances_weekly, demands_weekly, capacity_truck, num_wolves=100, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764759
Camión 1: [0, 3, 7, 2, 0]
Camión 2: [0, 6, 0]
Camión 3: [0, 8, 1, 5, 0]
Camión 4: [0, 12, 10, 9, 11, 0]
Camión 5: [0, 4, 0]
Tiempo de ejecución: 4.88019061088562 segundos


## RS

In [25]:
start = time.time()
perm, _ = simulated_annealing(distances_weekly, demands_weekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=10000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 85.16142483764759
Camión 1: [0, 1, 5, 8, 0]
Camión 2: [0, 7, 3, 2, 0]
Camión 3: [0, 9, 11, 12, 10, 0]
Camión 4: [0, 6, 0]
Camión 5: [0, 4, 0]
Tiempo de ejecución: 0.12074828147888184 segundos


In [26]:
start = time.time()
perm, _ = simulated_annealing(distances_weekly, demands_weekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 89.12268782744718
Camión 1: [0, 4, 5, 0]
Camión 2: [0, 8, 1, 0]
Camión 3: [0, 6, 0]
Camión 4: [0, 3, 2, 7, 0]
Camión 5: [0, 11, 12, 9, 10, 0]
Tiempo de ejecución: 0.0345766544342041 segundos


In [27]:
start = time.time()
perm, _ = simulated_annealing(distances_weekly, demands_weekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=200)
rutas, dist_total = construir_rutas_con_salida(perm, demands_weekly, capacity_truck, distances_weekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 86.87746526517708
Camión 1: [0, 2, 8, 7, 0]
Camión 2: [0, 1, 5, 3, 0]
Camión 3: [0, 11, 12, 10, 9, 0]
Camión 4: [0, 6, 0]
Camión 5: [0, 4, 0]
Tiempo de ejecución: 0.014360666275024414 segundos


# quincenal

## jaya

In [28]:
start = time.time()
perm, _ = jaya(distances_biweekly, demands_biweekly, capacity_truck)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 70.87147585756317
Camión 1: [0, 9, 1, 3, 4, 0]
Camión 2: [0, 6, 5, 0]
Camión 3: [0, 8, 0]
Camión 4: [0, 7, 0]
Camión 5: [0, 2, 0]
Tiempo de ejecución: 5.182040214538574 segundos


In [29]:
start = time.time()
perm, _ = jaya(distances_biweekly, demands_biweekly, capacity_truck, pop_size=100, max_iter=2000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 70.87147585756318
Camión 1: [0, 4, 3, 1, 9, 0]
Camión 2: [0, 8, 0]
Camión 3: [0, 2, 0]
Camión 4: [0, 7, 0]
Camión 5: [0, 6, 5, 0]
Tiempo de ejecución: 10.53414273262024 segundos


In [30]:
start = time.time()
perm, _ = jaya(distances_biweekly, demands_biweekly, capacity_truck,pop_size=600, max_iter=2000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

Distancia total: 70.87147585756317
Camión 1: [0, 9, 1, 3, 4, 0]
Camión 2: [0, 6, 5, 0]
Camión 3: [0, 8, 0]
Camión 4: [0, 7, 0]
Camión 5: [0, 2, 0]
Tiempo de ejecución: 61.70775532722473 segundos


## GW

In [ ]:
start = time.time()
perm, _ = gwo(distances_biweekly, demands_biweekly, capacity_truck)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

In [ ]:
start = time.time()
perm, _ = gwo(distances_biweekly, demands_biweekly, capacity_truck, num_wolves=50, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

In [ ]:
start = time.time()
perm, _ = gwo(distances_biweekly, demands_biweekly, capacity_truck, num_wolves=100, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

## RS

In [ ]:
start = time.time()
perm, _ = simulated_annealing(distances_biweekly, demands_biweekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=10000)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

In [ ]:
start = time.time()
perm, _ = simulated_annealing(distances_biweekly, demands_biweekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=500)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")

In [ ]:
start = time.time()
perm, _ = simulated_annealing(distances_biweekly, demands_biweekly, capacity_truck, T0=1000, cooling_rate=0.99, max_iter=200)
rutas, dist_total = construir_rutas_con_salida(perm, demands_biweekly, capacity_truck, distances_biweekly)
end = time.time()

print(f"Distancia total: {dist_total}")
for i, r in enumerate(rutas):
    print(f"Camión {i+1}: {r}")
print(f"Tiempo de ejecución: {end - start} segundos")